In [3]:
# === Interactive Peak Fitting Script for TOF-SIMS / Mass Spectra ===
"""
Title: Interactive Peak Fitting Tool
Author: Xiao Sun (xiao.sun@curtin.edu.au)
Version: 1.4.6
Date: 13-06-2025

Description:
Interactive Jupyter Notebook tool for peak fitting in TOF-SIMS/mass spectrometry data.
Supports Gaussian, Lorentzian, and PseudoVoigt fits with customizable symmetric/asymmetric options,
baseline correction, and smoothing.
Now includes button-controlled saving of figure and summary table based on the raw data filename.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.integrate import trapezoid
from lmfit.models import GaussianModel, LorentzianModel, PseudoVoigtModel, VoigtModel
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
import os

try:
    from lmfit.models import SkewedGaussianModel
except ImportError:
    SkewedGaussianModel = None

warnings.filterwarnings("ignore", category=UserWarning, module="uncertainties.core")

# === Load Spectral Data ===
txt_file = "2pc U_ROI3_5x5um_100pA_20kV_+ve.txt"
x_column = 'mass/charge (m/Q)'
y_column = 'Total (cts/TOF-Extraction)'
data = pd.read_csv(txt_file, sep='\t', comment='#')
data = data.dropna(subset=[x_column, y_column])
x = data[x_column].values
y = data[y_column].values

# === User Interface Widgets ===
fit_model = widgets.Dropdown(options=['gaussian', 'lorentzian', 'pseudovoigt'], value='lorentzian', description='Fit Model:')
symmetric_fit = widgets.Checkbox(value=True, description='Symmetric Peak')
use_baseline = widgets.Checkbox(value=True, description='Baseline Correction')
baseline_type = widgets.Dropdown(options=['average', 'linear', 'polynomial'], value='linear')
smoothing_window = widgets.IntSlider(value=11, min=3, max=51, step=2, description='Smooth Win:', layout=widgets.Layout(width='300px'))
smoothing_poly = widgets.IntSlider(value=3, min=1, max=5, step=1, description='Polyorder:', layout=widgets.Layout(width='250px'))

region_labels = ['235U', '238U', '235UO', '238UO', '235UO2', '238UO2']
peak_centers = {'235U': 235.0, '238U': 238.0, '235UO': 251.0, '238UO': 254.0, '235UO2': 267.0, '238UO2': 270.0}
range_width_widgets = {}
baseline_offset_widgets = {}

for label in region_labels:
    center_widget = widgets.FloatText(value=peak_centers[label], description=f'{label}:', layout=widgets.Layout(width='150px', margin='0 10px 0 0'), step=0.1, format='%.1f')
    width_widget = widgets.FloatText(value=1.0, description="Peak range: ±", layout=widgets.Layout(width='140px', margin='0 10px 0 0'), step=0.1, format='%.1f')
    offset_widget = widgets.FloatText(value=1.0, description="Baseline: ±", layout=widgets.Layout(width='140px', margin='0 10px 0 0'), step=0.05, format='%.1f')
    range_width_widgets[label] = (center_widget, width_widget)
    baseline_offset_widgets[label] = offset_widget

fit_button = widgets.Button(description="Fit and Calculate Ratios", layout=widgets.Layout(width='300px', height='30px'),
                            style={'button_color': 'lightblue', 'font_weight': 'bold', 'font_size': '15px'})
save_button = widgets.Button(description="Save Results", layout=widgets.Layout(width='200px', height='30px'))

ui = widgets.VBox([
    widgets.HBox([fit_model, symmetric_fit, use_baseline, baseline_type]),
    widgets.HBox([smoothing_window, smoothing_poly]),
    widgets.HTML("<b>Edit Peak Centers and Ranges</b><br><i>Baseline estimated from model at center ± baseline m/z offset</i>"),
    *[widgets.HBox([range_width_widgets[label][0], range_width_widgets[label][1], baseline_offset_widgets[label]]) for label in region_labels],
    widgets.HBox([fit_button, save_button])
])

fig = None
df_summary = None

def get_model(model_type, symmetric=True):
    if model_type == 'gaussian':
        return GaussianModel() if symmetric else (SkewedGaussianModel() if SkewedGaussianModel else GaussianModel())
    elif model_type == 'lorentzian':
        return LorentzianModel() if symmetric else VoigtModel()
    elif model_type == 'pseudovoigt':
        return PseudoVoigtModel() if symmetric else VoigtModel()
    else:
        raise ValueError("Unsupported model type")

def fit_peak_custom_baseline(x_all, y_all, model_type, mz_range, baseline_offset, symmetric, use_baseline, baseline_mode):
    mask_peak = (x_all >= mz_range[0]) & (x_all <= mz_range[1])
    x_peak = x_all[mask_peak]
    y_peak = y_all[mask_peak]

    if len(x_peak) < 5:
        raise ValueError("Peak range too narrow or no data points available.")

    model = get_model(model_type, symmetric=symmetric)
    center_guess = (mz_range[0] + mz_range[1]) / 2
    height_guess = np.max(y_peak)
    sigma_guess = (mz_range[1] - mz_range[0]) / 4
    amplitude_guess = height_guess * sigma_guess * np.sqrt(2 * np.pi)
    params = model.make_params(center=center_guess, amplitude=amplitude_guess, sigma=sigma_guess)

    initial_result = model.fit(y_peak, params, x=x_peak)

    if use_baseline:
        if baseline_mode == 'average':
            left_val = np.interp(center_guess - baseline_offset, x_peak, initial_result.best_fit)
            right_val = np.interp(center_guess + baseline_offset, x_peak, initial_result.best_fit)
            baseline = np.mean([left_val, right_val])
            y_corr = y_peak - baseline
        elif baseline_mode == 'linear':
            center = (mz_range[0] + mz_range[1]) / 2
            left_mask = (x_all >= center - baseline_offset) & (x_all < center - baseline_offset / 2)
            right_mask = (x_all > center + baseline_offset / 2) & (x_all <= center + baseline_offset)
            x_baseline = np.concatenate([x_all[left_mask], x_all[right_mask]])
            y_baseline = np.concatenate([y_all[left_mask], y_all[right_mask]])
            p = np.polyfit(x_baseline, y_baseline, deg=1)
            baseline_line = np.polyval(p, x_peak)
            y_corr = y_peak - baseline_line
        elif baseline_mode == 'polynomial':
            p = np.polyfit(x_peak, y_peak, deg=2)
            baseline_line = np.polyval(p, x_peak)
            y_corr = y_peak - baseline_line
        else:
            y_corr = y_peak
    else:
        y_corr = y_peak

    final_result = model.fit(y_corr, model.make_params(center=center_guess, amplitude=amplitude_guess, sigma=sigma_guess), x=x_peak)
    area = final_result.params['amplitude'].value
    residuals = y_corr - final_result.best_fit
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_corr - np.mean(y_corr))**2)
    r_squared = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan

    return final_result, x_peak, y_peak, y_corr, area, residuals, r_squared

def run_fitting(b):
    global df_summary, fig
    clear_output(wait=True)
    display(ui)

    if smoothing_window.value >= len(y):
        print("Error: Smoothing window too large for data length.")
        return

    y_smooth = savgol_filter(y, smoothing_window.value, smoothing_poly.value)

    regions = {
        label: (
            range_width_widgets[label][0].value - range_width_widgets[label][1].value,
            range_width_widgets[label][0].value + range_width_widgets[label][1].value
        )
        for label in region_labels
    }
    offsets = {label: baseline_offset_widgets[label].value for label in region_labels}
    ratio_groups = [('235U', '238U'), ('235UO', '238UO'), ('235UO2', '238UO2')]

    areas = {}
    r2_values = {}
    results = {}

    fig, axes = plt.subplots(3, 2, figsize=(14, 12))
    axes = axes.flatten()

    for i, label in enumerate(region_labels):
        mz_range = regions[label]
        baseline_offset = offsets[label]

        try:
            result, x_fit, y_fit, y_corr, area, residuals, r2 = fit_peak_custom_baseline(
                x, y_smooth, fit_model.value, mz_range, baseline_offset, symmetric_fit.value, use_baseline.value, baseline_type.value
            )
        except Exception as e:
            print(f"Error fitting {label}: {e}")
            continue

        areas[label] = area
        r2_values[label] = r2
        results[label] = result

        ax = axes[i]
        ax.plot(x_fit, y_fit, label='Smoothed', color='blue')
        if use_baseline.value:
            ax.plot(x_fit, y_corr, label='Corrected', color='purple')
        ax.plot(x_fit, result.best_fit, 'r--', label='Fit')
        ax.plot(x_fit, residuals, 'k:', label='Fitting deviation')
        ax.set_title(f"{label} (R² = {r2:.4f})")
        ax.legend()
        ax.grid(True)

    plt.tight_layout()
    plt.show()

    def calc_ratio(a1, a2):
        return a1 / (a1 + a2) if (a1 + a2) > 0 else np.nan

    summary_rows = []
    model_name = fit_model.value.capitalize()

    for label in region_labels:
        area_val = areas.get(label, np.nan)
        summary_rows.append({
            "Ions": label,
            "Model": model_name,
            "Area": area_val,
            "R²": r2_values.get(label, np.nan),
            "Isotope Ratio": ""
        })

    for a1, a2 in ratio_groups:
        r = calc_ratio(areas.get(a1, 0), areas.get(a2, 0))
        for row in summary_rows:
            if row["Ions"] == a1:
                row["Isotope Ratio"] = f"{r:.4f}"

    df_summary = pd.DataFrame(summary_rows, columns=["Ions", "Model", "Area", "R²", "Isotope Ratio"])
    df_summary.index = np.arange(1, len(df_summary) + 1)
    display(df_summary)

def save_results(b):
    global df_summary, fig
    if df_summary is None or fig is None:
        print("No results to save. Please run the fitting first.")
        return
    base_name = os.path.splitext(os.path.basename(txt_file))[0]
    fig.savefig(f"{base_name}_fit.png", dpi=300)
    df_summary.to_csv(f"{base_name}_summary.csv", index=False)
    print(f"Saved: {base_name}_fit.png and {base_name}_summary.csv")

fit_button.on_click(run_fitting)
save_button.on_click(save_results)
display(ui)
